In [ ]:
%load_ext autoreload

%autoreload 2

# Align IF to imc

This will fine-align the coarsly IMC to IF images in two steps:
- Align each slidescan scene to the matching IMC panorama images
- Crop the Slidescan scene around the estimated spheroid crop location and fine-align IF and IMC on the spheroid level.

In [ ]:
import workflow.scripts.utils_alignment.library_java as libj
import workflow.scripts.utils_alignment.library as lib
import workflow.scripts.utils_alignment.alignment as alignment
import pandas as pd
import numpy as np
import pathlib
from tqdm import tqdm

In [ ]:
import workflow.scripts.utils_alignment.variables as V

In [ ]:
Cin = snakemake.input
Cout = snakemake.output
Cparams = snakemake.params

In [ ]:
for fol in [Cout.fol_out_trakem2, Cout.fol_spherecrop_aligned]:
    pathlib.Path(Cout.fol_out_trakem2).mkdir(parents=True, exist_ok=True)

In [ ]:
Cout.fol_out_trakem2 = pathlib.Path(Cout.fol_out_trakem2).resolve()

In [ ]:
Cout.fol_spherecrop_aligned = pathlib.Path(Cout.fol_spherecrop_aligned ).resolve()
Cout.fol_spherecrop_aligned.mkdir(parents=True, exist_ok=True)
Cin.fol_cp_full = pathlib.Path(Cin.fol_cp_full).resolve()
Cin.fol_out_imgs = pathlib.Path(Cin.fol_out_imgs).resolve()
Cin.fol_crop = pathlib.Path(Cin.fol_crop).resolve()

In [ ]:

dat_cords = pd.read_csv(Cin.fn_cords)

dat_scenecords = pd.read_csv(Cin.fn_scenecords)



In [ ]:
dat_cropmeta = pd.read_csv(Cin.fn_cropmeta)
dat_cropmeta = dat_cropmeta.merge(dat_cords)

Get metadata for scene

In [ ]:
transf_if_imc_params = np.loadtxt(Cin.fn_transf_ifslide_imcslide)

In [ ]:
at_if2imc = libj.params_to_affinetransform(transf_if_imc_params)

In [ ]:
# Follow: https://imagej.net/TrakEM2_Scripting

Align IF to IMC using a rigid transform

In [ ]:
params_coarse, params_fine = alignment.get_scene_alignparams()

In [ ]:
params_align = [params_coarse, params_fine]

In [ ]:
%%time
for idx, row in dat_cords[[V.SLIDESCAN, V.SCENEID]].drop_duplicates().iterrows():
    alignment.align_scene(row[V.SLIDESCAN], row[V.SCENEID],
                dat_scenecords,
                dat_cords,
                transf_scene2imc=at_if2imc,
                fol_trakem2=Cout.fol_out_trakem2,
                fol_imgs_scene=Cin.fol_out_imgs,
                fol_imgs_imc=Cin.fol_cp_full,
                align_params=params_align,
                channel_slide=Cparams.channel_slide,
                desc='v1')
    print(list(filter(lambda x: 'corresponding features with an average displacement of' in x, filter(lambda x: x != '\n', libj.logger.out))))
    libj.logger.clear_output()

Save project

Now crop the 'spheres' using the cellprofiler file names, both in IF as well as IMC. then fine align the spheres and save out the aligned images.

In [ ]:
import re
import skimage.io as skio
import pathlib
import matplotlib.pyplot as plt

In [ ]:
%%time
rlist = []
for idx, row in dat_cords[[V.SLIDESCAN, V.SCENEID]].drop_duplicates().iterrows():
    slidescan, sceneid = row[V.SLIDESCAN], row[V.SCENEID]
    d=alignment.get_params_from_project(Cout.fol_out_trakem2/ V.TPL_SLIDESCENE_ALIGN.format(slide=slidescan, scene=sceneid,
                                                                             desc='v1'))
    dat= pd.DataFrame(d)
    dat[V.SLIDESCAN] = slidescan
    dat[V.SCENEID] = sceneid
    rlist.append(dat)
    
dat_params_scenealign = pd.concat(rlist) 

In [ ]:
%%time
dat_scene_rcrop = dat_params_scenealign.merge(dat_cropmeta).apply(alignment.get_scene_rough_imccrop, axis=1)

Check which scenes have obviously not been matched correctly

In [ ]:
dat_scene_rcrop.loc[(dat_scene_rcrop[V.CROPY] < 0) | (dat_scene_rcrop[V.CROPX] < 0) ,:]

In [ ]:
dat_scene_rcrop.to_csv(Cout.fn_scene_rcrop, index=False)

-> Only 1 scene which is also blurry & out of focus, quite acceptable!

In [ ]:
%%time
alignment.crop_scene(dat_scene_rcrop, Cin.fol_out_imgs, Cin.fol_out_imgs, 'rcrop', channel=Cparams.channel_slide)

In [ ]:
dat_scene_rcrop = pd.read_csv(Cout.fn_scene_rcrop)

In [ ]:
paramAffine = alignment.get_crop_alignparms()

In [ ]:
libj.logger.clear_output()
for (croppath), d in tqdm(dat_scene_rcrop.groupby(by=[V.CROPPATH])):
    alignment.align_crops(croppath, dat_scene_rcrop, dat_cropmeta,
                          Cout.fol_out_trakem2, Cin.fol_out_imgs, Cin.fol_crop, align_params=[paramAffine], cropdesc='rcrop', channel_slide=Cparams.channel_slide)
    print(list(filter(lambda x: 'displacement' in x, filter(lambda x: x != '\n', libj.logger.out))))
    libj.logger.clear_output()

In [ ]:
%%time
rlist = []
for idx, row in dat_scene_rcrop[[V.CROPPATH]].drop_duplicates().iterrows():
    croppath = row[V.CROPPATH]
    try:
        d=alignment.get_params_from_project(Cout.fol_out_trakem2/ V.TPL_SLIDESCENE_CROP_ALIGN.format(
                crop_path=croppath, cropdesc='rcrop', desc='v1'))
    except AttributeError:
        continue
    dat= pd.DataFrame(d)
    dat[V.CROPPATH] = croppath 
    rlist.append(dat)
    


In [ ]:
dat_params_fine = pd.concat(rlist).reset_index(drop=True)

In [ ]:
%%time

(dat_params_fine.merge(dat_cropmeta, on=V.CROPPATH)
 .apply(alignment.crop_scene_fine, axis=1, fol_imgs=Cin.fol_out_imgs,
        fol_out=Cout.fol_spherecrop_aligned, scale=2, channels=[0]))

Visualize a random example

In [ ]:
import skimage.io as skio
import skimage.transform as sktransf
import matplotlib.pyplot as plt

In [ ]:
d= dat_params_fine.merge(dat_cropmeta, on=V.CROPPATH).iloc[20]
transf = alignment.get_scene_fine_croptransf(d, scale=2)

In [ ]:

fn = Cin.fol_out_imgs/ V.TPL_SLIDESCENE_CROP_IMG.format(
            crop_path=d[V.CROPPATH], channel=Cparams.channel_slide, cropdesc='rcrop', desc='v1')
img = skio.imread(fn)

In [ ]:


fn = Cin.fol_crop/ (d[V.CROPPATH]+'.tiff')
img_crop = skio.imread(fn)
img_crop = sktransf.rescale(img_crop, 2)
print(fn)

In [ ]:

img_t = sktransf.warp(img, transf.inverse, output_shape=np.array(img_crop.shape))

In [ ]:
plt.imshow(img_t)

In [ ]:
plt.imshow(np.sqrt(img_crop))

In [ ]:
im = np.zeros(list(img_t.shape)+[3])
im[:,:,0] = img_t/img_t.max()
im[:,:,1] = np.sqrt(img_crop)/np.sqrt(img_crop).max()
im[im>1] =1

In [ ]:

fn = Cin.fol_out_imgs/ V.TPL_SLIDESCENE_CROP_IMG.format(
            crop_path=d[V.CROPPATH], channel=Cparams.channel_slide, cropdesc='rcrop', desc='v1')
img = skio.imread(fn)

In [ ]:


fn = Cin.fol_crop/ (d[V.CROPPATH]+'.tiff')
img_crop = skio.imread(fn)
img_crop = sktransf.rescale(img_crop, 2)
print(fn)

In [ ]:

img_t = sktransf.warp(img, transf.inverse, output_shape=np.array(img_crop.shape))

In [ ]:
plt.imshow(img_t)

In [ ]:
plt.imshow(np.sqrt(img_crop))

In [ ]:
im = np.zeros(list(img_t.shape)+[3])
im[:,:,0] = img_t/img_t.max()
im[:,:,1] = np.sqrt(img_crop)/np.sqrt(img_crop).max()
im[im>1] =1

In [ ]:
plt.imshow(im)